In [1]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from bs4 import BeautifulSoup
import re
from twilio.rest import Client
import sys

drugs = ['Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine', 'Hydromorphone', 'Opiate']
myfile = pd.read_csv("drugDeath.csv")

TableHeader=list()

for row in myfile:
    TableHeader.append(row)
Druglist=[]
for i in range (20,35):
    Druglist.append(TableHeader[i])
    
    
# Account Sid and Auth Token from twilio.com/console
# http://twil.io/secure
account_sid = 'ACf6879452cddae5d29cbb5348278fc41f'
auth_token = '0ba1e162e8366dbcecc129c5b035a6ef'
client = Client(account_sid, auth_token)

message = client.messages.create(
                              body='Hi Professor Thai, This is a message from the python code by KSSMG! Here is a link to our dataset: https://catalog.data.gov/dataset/accidental-drug-related-deaths-january-2012-sept-2015/resource/44580a89-a260-4844-8ead-35736f395389',
                              from_='+17042094792',
                              to='+13012460510'
                          )

print(message.sid)
    
while True:
#convert list to string and generate
    unique_string=(" ").join(Druglist)
    wordcloud = WordCloud(width = 1000, height = 500).generate(unique_string)
    plt.figure(figsize=(15,8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.savefig("Druglist"+".png", bbox_inches='tight')
    plt.show()
    plt.close()

    
    SexDict={"F":"Female","f":"Female","female":"Female","Female":"Female", "Male":"Male", "M":"Male","m":"Male","male":"Male"}
    while True:
        gender=input("Input the gender (male - female - both): ").strip().lower()
        if gender in SexDict: 
            break
        elif gender == "both" or gender == 'b':
            temp1 = "male"
            temp2 = "female"
            break
        else:
            print("That is not a valid choice.\n")
            continue
                
            
    while True:
        Drug=input("Enter the name of the drug (using the WordCloud):  ").strip().capitalize()
        if Drug in Druglist:
            break
        else:
            print("That is not a valid choice.\n")
            continue
                
    DG=[]
    index= Druglist.index(Drug)+20
    DG=myfile.iloc[:,[4,index]]

    kkk=0
    count=0
    m=0
    f=0
    for kkk in range(1, len(DG)):
        if gender in SexDict:
            if DG.iloc[kkk,0]==SexDict[gender]:
                if DG.iloc[kkk,1]=="Y":
                    count+=1
                    continue
        elif gender == "both":
            if DG.iloc[kkk,0]==SexDict[temp1]:
                if DG.iloc[kkk,1]=="Y":
                    m+=1
                    continue
            elif DG.iloc[kkk,0]==SexDict[temp2]:
                if DG.iloc[kkk,1]=="Y":
                    f+=1
                    continue
    if gender in SexDict:
        print(str(count)+ " " + SexDict[gender] +" died from "+ Drug, "\n")
    elif gender == "both":
        print(str(m)+ " " + SexDict[temp1] +" died from "+ Drug, "\n")
        print(str(f)+ " " + SexDict[temp2] +" died from "+ Drug, "\n")

    while True:    
        if gender in SexDict:
            choice = input("Would you like to see the data for all the drugs for " + gender + "? (yes or no): ").strip().lower()
        
            if (choice == "yes" and (gender == 'female' or gender == 'f')):
                choice = input("Would you like a pie chart or bar chart? (pie, bar or both): ").strip().lower()
                if (choice == "pie"):
                    x = np.char.array(['Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine', 'Hydromorphone', 'Opiate'   ]) #labels
                    y = np.array([539, 381, 465, 204, 35, 274, 55, 463, 159, 48, 46, 17, 9, 32]) #values
                    colors = ['yellowgreen','red','gold','lightskyblue','white','lightcoral','blue','pink', 'darkgreen','yellow','grey','violet','magenta','cyan']
                    porcent = 100.*y/y.sum()

                    patches, texts = plt.pie(y, colors=colors, startangle=90, radius=1.2)
                    labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(x, porcent)]

                    sort_legend = True
                    if sort_legend:
                        patches, labels, dummy =  zip(*sorted(zip(patches, labels, y), key=lambda x: x[2], reverse=True))
        
                        plt.legend(patches, labels, loc='right', bbox_to_anchor=(-0.1, 1.), fontsize=8)
                        plt.savefig("femalepiechart.png")
                        plt.title("Female Drug Deaths")

                        plt.show()
              
                elif (choice == "bar"):
                    df = pd.DataFrame({'Drugs':['Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine', 'Hydromorphone', 'Opiate'], 'Deaths':[539, 381, 465, 204, 35, 274, 55, 463, 159, 48, 46, 17, 9, 32]})
                    ax = df.plot.bar(x='Drugs', y='Deaths', rot=90)
                  
                    plt.savefig("femalebarchart.png")
                    plt.title("Female Drug Deaths")
                    plt.figure( figsize=(20,10), facecolor='w')
                    plt.show()
                elif (choice == "both" or choice == "b"):
                    #pie
                    x = np.char.array(['Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine', 'Hydromorphone', 'Opiate'   ]) #labels
                    y = np.array([539, 381, 465, 204, 35, 274, 55, 463, 159, 48, 46, 17, 9, 32]) #values
                    colors = ['yellowgreen','red','gold','lightskyblue','white','lightcoral','blue','pink', 'darkgreen','yellow','grey','violet','magenta','cyan']
                    porcent = 100.*y/y.sum()

                    patches, texts = plt.pie(y, colors=colors, startangle=90, radius=1.2)
                    labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(x, porcent)]

                    sort_legend = True
                    if sort_legend:
                        patches, labels, dummy =  zip(*sorted(zip(patches, labels, y), key=lambda x: x[2], reverse=True))
        
                        plt.legend(patches, labels, loc='right', bbox_to_anchor=(-0.1, 1.), fontsize=8)
                        plt.savefig("femalepiechart.png")
                        plt.title("Female Drug Deaths")

                        plt.show()

                    #bar
                    df = pd.DataFrame({'Drugs':['Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine', 'Hydromorphone', 'Opiate'], 'Deaths':[539, 381, 465, 204, 35, 274, 55, 463, 159, 48, 46, 17, 9, 32]})
                    ax = df.plot.bar(x='Drugs', y='Deaths', rot=90)

                    plt.savefig("femalebarchart.png")
                    plt.title("Female Drug Deaths")
                    plt.figure( figsize=(20,10), facecolor='w')
                    plt.show()
                else:
                    break
              
            elif (choice == "yes" and (gender == 'male' or gender == 'm')):
                choice = input("Would you like a pie chart or bar chart? (pie, bar or both): ").strip().lower()
                if (choice == "pie"):
                    x = np.char.array(['Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine', 'Hydromorphone', 'Opiate'   ]) #labels
                    y = np.array([1989, 1138, 1758, 402, 73, 971, 62, 878, 314, 111, 83, 21, 15 , 56]) #values
                    colors = ['yellowgreen','red','gold','lightskyblue','white','lightcoral','blue','pink', 'darkgreen','yellow','grey','violet','magenta','cyan']
                    porcent = 100.*y/y.sum()

                    patches, texts = plt.pie(y, colors=colors, startangle=90, radius=1.2)
                    labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(x, porcent)]

                    sort_legend = True
                    if sort_legend:
                        patches, labels, dummy =  zip(*sorted(zip(patches, labels, y), key=lambda x: x[2], reverse=True))

                    plt.legend(patches, labels, loc='right', bbox_to_anchor=(-0.1, 1.), fontsize=8)
                    plt.savefig("malepiechart.png", bbox_inches="tight")
                    plt.title("Male Drug Deaths")

                    plt.show()  
                  
                elif(choice == "bar"):
                    df = pd.DataFrame({'Drugs':['Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine', 'Hydromorphone', 'Opiate'], 'Deaths':[1989, 1138, 1758, 402, 73, 971, 62, 878, 314, 111, 83, 21, 15 , 56]})
                    ax = df.plot.bar(x='Drugs', y='Deaths', rot=90)
                    plt.savefig("malebarchart.png")
                    plt.title("Male Drug Deaths")

                    plt.figure( figsize=(20,10), facecolor='w')
                    
                    plt.show()
                elif(choice == "both" or choice == "b"):
                    #pie
                    x = np.char.array(['Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine', 'Hydromorphone', 'Opiate'   ]) #labels
                    y = np.array([1989, 1138, 1758, 402, 73, 971, 62, 878, 314, 111, 83, 21, 15 , 56]) #values
                    colors = ['yellowgreen','red','gold','lightskyblue','white','lightcoral','blue','pink', 'darkgreen','yellow','grey','violet','magenta','cyan']
                    porcent = 100.*y/y.sum()

                    patches, texts = plt.pie(y, colors=colors, startangle=90, radius=1.2)
                    labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(x, porcent)]

                    sort_legend = True
                    if sort_legend:
                        patches, labels, dummy =  zip(*sorted(zip(patches, labels, y), key=lambda x: x[2], reverse=True))

                    plt.legend(patches, labels, loc='right', bbox_to_anchor=(-0.1, 1.), fontsize=8)
                    plt.savefig("malepiechart.png", bbox_inches="tight")
                    plt.title("Male Drug Deaths")

                    plt.show()

                    #bar
                    df = pd.DataFrame({'Drugs':['Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine', 'Hydromorphone', 'Opiate'], 'Deaths':[1989, 1138, 1758, 402, 73, 971, 62, 878, 314, 111, 83, 21, 15 , 56]})
                    ax = df.plot.bar(x='Drugs', y='Deaths', rot=90)
                    plt.savefig("malebarchart.png")
                    plt.title("Male Drug Deaths")

                    plt.figure( figsize=(20,10), facecolor='w')
                    
                    plt.show()
                else:
                    break

            elif (choice == "no"):
                break
      
        elif gender == "both" or gender == "b":
            choice = input("Would you like to see the data for all the drugs for both genders? (yes or no): ").strip().lower()
            
            if (choice == "yes" or choice == 'y'):
                choice = input("Would you like a line chart or bar chart? (line, bar or both): ").strip().lower()
                if (choice == "bar"):
                    fig = go.Figure(data=[
                      go.Bar(name='Male', x=drugs, y=[1989, 1138, 1758, 402, 73, 971, 62, 878, 314, 111, 83, 21, 15 , 56]),
                      go.Bar(name='Female', x=drugs, y=[539, 381, 465, 204, 35, 274, 55, 463, 159, 48, 46, 17, 9, 32])
                    ])
                    
                    fig.update_layout(barmode='group')
                    fig.show()
                elif(choice == "line"):
                    fig = make_subplots(specs=[[{"secondary_y": True}]])
                    
                    fig.add_trace(
                        go.Scatter(x=drugs, y=[1989, 1138, 1758, 402, 73, 971, 62, 878, 314, 111, 83, 21, 15 , 56], name="male"),
                        secondary_y=False,
                      )
                    
                    fig.add_trace(
                        go.Scatter(x=drugs, y=[539, 381, 465, 204, 35, 274, 55, 463, 159, 48, 46, 17, 9, 32], name="female"),
                        secondary_y=True,
                    )
                    
                    fig.update_layout(
                    title_text="Double Y Axis For Male and Female"
                    )


                    fig.update_xaxes(title_text="Drug Type")

                    fig.update_yaxes(title_text="Male Deaths", secondary_y=False)
                    fig.update_yaxes(title_text="Female Deaths", secondary_y=True)

                    fig.show()

                elif (choice == "both" or choice == "b"):
                    #bar
                    fig = go.Figure(data=[
                      go.Bar(name='Male', x=drugs, y=[1989, 1138, 1758, 402, 73, 971, 62, 878, 314, 111, 83, 21, 15 , 56]),
                      go.Bar(name='Female', x=drugs, y=[539, 381, 465, 204, 35, 274, 55, 463, 159, 48, 46, 17, 9, 32])
                    ])
                    
                    fig.update_layout(barmode='group')
                    fig.show() 

                    #line
                    fig = make_subplots(specs=[[{"secondary_y": True}]])
                    
                    fig.add_trace(
                        go.Scatter(x=drugs, y=[1989, 1138, 1758, 402, 73, 971, 62, 878, 314, 111, 83, 21, 15 , 56], name="male"),
                        secondary_y=False,
                      )
                    
                    fig.add_trace(
                        go.Scatter(x=drugs, y=[539, 381, 465, 204, 35, 274, 55, 463, 159, 48, 46, 17, 9, 32], name="female"),
                        secondary_y=True,
                    )
                    
                    fig.update_layout(
                    title_text="Double Y Axis For Male and Female"
                    )


                    fig.update_xaxes(title_text="Drug Type")

                    fig.update_yaxes(title_text="Male Deaths", secondary_y=False)
                    fig.update_yaxes(title_text="Female Deaths", secondary_y=True)

                    fig.show()

                else:
                    break

            elif choice == "no" or choice == 'n':
                break
            else:
                print("That is not a valid choice\n")

    while True:
        choice = input("Would you like to read additional information about " + Drug + "? (yes or no) ").strip().lower()
        if choice == "yes" or choice == "y":
            
            if Drug == "Amphet":
                urllink = "https://en.wikipedia.org/wiki/Amphetamine"
                file_contents = "Amphetamine.txt"
                pass
            elif Drug == "Benzodiazepine":
                urllink = "https://en.wikipedia.org/wiki/Benzodiazepine"
                file_contents = "Benzodiazepine.txt"
                pass
            elif Drug == "Cocaine":
                urllink = "https://en.wikipedia.org/wiki/Cocaine"
                file_contents = "Cocaine.txt"
                pass
            elif Drug == "Ethanol":
                urllink = "https://en.wikipedia.org/wiki/Ethanol"
                file_contents = "Ethanol.txt"
                pass
            elif Drug == "Fentanyl":
                urllink = "https://en.wikipedia.org/wiki/Fentanyl"
                file_contents = "Fentanyl.txt"
                pass
            elif Drug == "Heroin":
                urllink = "https://en.wikipedia.org/wiki/Heroin"
                file_contents = "Heroin.txt"
                pass
            elif Drug == "Hydrocodone":
                urllink = "https://en.wikipedia.org/wiki/Hydrocodone"
                file_contents = "Hydrocodone.txt"
                pass
            elif Drug == "Hydromorphone":
                urllink = "https://en.wikipedia.org/wiki/Hydromorphone"
                file_contents = "Hydromorphone.txt"
                pass
            elif Drug == "Methadone":
                urllink = "https://en.wikipedia.org/wiki/Methadone"
                file_contents = "Methadone.txt"
                pass
            elif Drug == "Morphine":
                urllink = "https://en.wikipedia.org/wiki/Morphine"
                file_contents = "Morphine.txt"
                pass
            elif Drug == "Opiate":
                urllink = "https://en.wikipedia.org/wiki/Opiate"
                file_contents = "Opiate.txt"
                pass
            elif Drug == "Oxycodone":
                urllink = "https://en.wikipedia.org/wiki/Oxycodone"
                file_contents = "Oxycodone.txt"
                pass
            elif Drug == "Oxymorphone":
                urllink = "https://en.wikipedia.org/wiki/Oxymorphone"
                file_contents = "Oxymorphone.txt"
                pass
            elif Drug == "Tramad":
                urllink = "https://en.wikipedia.org/wiki/Tramadol"
                file_contents = "Tramadol.txt"
                pass

            #Taking the URL input and validating using Regex

            URL = urllink
            urlRegex = re.compile(r"^https\:\/\/([\w\.]+)wikipedia.org\/wiki\/([\w]+\_?)+")
            mo = urlRegex.search(URL)
            if mo == None:
                print("Wrong URL entered. Make sure to enter a valid Wikipedia URL. Make sure to add https:// before the URL if you forgot.")
                exit()

            #Requesting the HTML and making the BeautifulSoup object
            req = requests.get(mo.group())
            soup = BeautifulSoup(req.text, "lxml")

            #Validating if the site has content
            if soup.find("p").text.strip() == "Other reasons this message may be displayed:":
                print("This Wikipedia site does not exists.\n")
                exit()

            #Retriving and printing the page title
            page_title = soup.find("h1", class_="firstHeading").text
            print(f"\n---{page_title}---\n")

            #Making the text file to save the text data
            f = open(f"{page_title}.txt", "w", encoding="utf-8")
            f.write(f"//{mo.group()}\n---{page_title}---\n\n")

            #Topics to avoid
            exclude = ["See also", "References", "Sources", "Further reading", "External links"]

            #Scraping the site for headings and paragraphs
            for info in soup.descendants:
                if info.name == "span":
                    try:
                        if info["class"][0] == "mw-headline":
                            headline = info.get_text()
                            if headline not in exclude:
                                print(f"{headline}:\n") #Printing the heading
                                f.write(f"\n{headline}:\n\n")
                    except KeyError: #try except block to handle BS KeyError
                        pass
                elif info.name == "p":
                    para = info.get_text()
                    print(f"{para}") #Printing the paragraph
                    f.write(f"{para}")

            f.close() #Closing the file

            file = open(file_contents, encoding="utf8").read()

            wordcloud = WordCloud(#font_path = r'C:\Windows\Fonts\Verdana.ttf',
                                    stopwords = STOPWORDS,
                                    background_color = 'white',
                                    width=1600, height=800
                                    ).generate(file)

            plt.figure( figsize=(20,10), facecolor='w')
            plt.imshow(wordcloud)
            plt.axis('off')
            plt.show()
            break
        elif choice == "no":
            break
        else:
            print("That is not a valid choice\n")

    choice = input("Do you want to keep searching? (yes or no) ").strip().lower()

    if choice == "yes" or choice == "y":
        continue
    else:
        print("Thanks for using the program!")
        break
    


    


TwilioRestException: HTTP 401 error: Unable to create record: Authenticate

# New Section

In [ ]:
! pip install twilio